In [ ]:
!nvidia-smi

In [ ]:
!pip install extra-keras-datasets

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt 
import numpy as np 
from extra_keras_datasets import emnist

In [ ]:
(x_train, y_train),(x_test_ori, y_test) = emnist.load_data(type='byclass')

In [ ]:
print(len(y_train))
print(len(y_test))

In [ ]:
plt.imshow(x_test_ori[1000], cmap="gray") # Import the image
plt.show() # Plot the image

In [ ]:
# reshape to be [samples][width][height][channels]
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32')
x_test = x_test_ori.reshape((x_test_ori.shape[0], 28, 28, 1)).astype('float32')

In [ ]:
# normalize inputs from 0-255 to 0-1
x_train = x_train / 255
x_test = x_test / 255
# one hot encode outputs for the loss funtion
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
# define the 3-layer CNN model
def cnn_model(dpout, opt, first_hidden, second_hidden, third_hidden, first_kernel, second_kernel, third_kernel, loss_f):
	# create model
    model = Sequential()
    model.add(Conv2D(first_hidden, (first_kernel, first_kernel), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(second_hidden, (second_kernel, second_kernel), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(third_hidden, (third_kernel, third_kernel), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(dpout))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
	# Compile model
    model.compile(loss=loss_f, optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
def plot_line_all(title, x_label, y_label, inputs, legends):
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    for inp in inputs:
        plt.plot(inp)
    plt.legend(legends, bbox_to_anchor=(1.04,1), loc="upper left")
    plt.grid()
    plt.show()

In [ ]:
# hyperparameters = [[0.2, 75, 75, 75, 5, 3, 3, 0.0001, 128, 'categorical_crossentropy'],
#                    [0.2, 90, 90, 90, 5, 3, 3, 0.0001, 128, 'categorical_crossentropy'],
#                    [0.2, 100, 100, 100, 5, 3, 3, 0.0001, 128, 'categorical_crossentropy']]
# hyperparameters = [[0.2, 100, 100, 100, 3, 3, 3, 0.0001, 128, 'categorical_crossentropy'],
#                    [0.2, 100, 100, 100, 5, 3, 3, 0.0001, 128, 'categorical_crossentropy'],
#                    [0.2, 100, 100, 100, 5, 5, 3, 0.0001, 128, 'categorical_crossentropy']]
# hyperparameters = [[0.2, 100, 100, 100, 5, 5, 3, 0.001, 128, 'categorical_crossentropy'],
#                    [0.2, 100, 100, 100, 5, 5, 3, 0.0001, 128, 'categorical_crossentropy']]
hyperparameters = [[0.2, 100, 100, 100, 5, 5, 3, 0.0001, 256, 'categorical_crossentropy']]
i = 0
history = []
strhyp = []
for dpout, hid1, hid2, hid3, ker1, ker2, ker3, lr, bs, ls in hyperparameters:
    keras.backend.clear_session()
    opt = keras.optimizers.Adam(learning_rate=lr)
    model = cnn_model(dpout, opt, hid1, hid2, hid3, ker1, ker2, ker3, ls)
    history.append(model.fit(x_train, y_train, validation_split=0.2, epochs=20, batch_size=bs))
    i += 1
    strhyp.append("dp: " + str(dpout) + " hid1: " + str(hid1) + " hid2: " + str(hid2) + " hid3: " + str(hid3) + " ker1: " + str(ker1) + " ker2: " + str(ker2) + " ker3: " + str(ker3) + " lr: " + str(lr) + " bs: " + str(bs) + " loss: " + ls)
    
    tloss, tacc = model.evaluate(x=x_test, y=y_test)
    print('\nTest accuracy:', tacc, ' Test loss:', tloss)

acc = []    
loss = []
for his in history:
    acc.append(his.history['val_accuracy'])
    loss.append(his.history['val_loss'])

# summarize history for accuracy
plot_line_all('model accuracy', 'epoch', 'accuracy', acc, strhyp)
# summarize history for loss
plot_line_all('model loss', 'epoch', 'loss', loss, strhyp)

In [ ]:
# Evaluate the model performance
test_loss, test_acc = model.evaluate(x=x_test, y=y_test)
# Print out the model accuracy 
print('\nTest accuracy:', test_acc)

In [ ]:
def mapping_res(num):
    if num < 10:
        print(chr(num + 48))
    elif num < 36:
        print(chr(num + 55))
    else:
        print(chr(num + 62))
        
def predict_and_compare(original, transform, idx):
    predictions = model.predict([transform])
    print("\nPREDICTION : ")
    mapping_res(np.argmax(predictions[idx]))
    print("\nTEST DATA : ")
    plt.imshow(original[idx], cmap="gray") 
    plt.show() 

In [ ]:
input = 3111
predict_and_compare(x_test_ori, x_test, input)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#generate and save math 343 data into mnist format
from PIL import Image
import os
import numpy as np

file_path = "drive/MyDrive/cnn_hand2txt/math_343_sample/"    
vector_images = []

for _, file in enumerate(os.listdir(file_path)):
    image = Image.open(file_path + file)
    image_array = np.array(image)
    vector_images.append(image_array)        
np.savez("drive/MyDrive/cnn_hand2txt/mnistlikemathdataset.npz",math_data=vector_images) 

In [ ]:
path = "drive/MyDrive/cnn_hand2txt/mnistlikemathdataset.npz"
with np.load(path) as dt:
    #load math data as test_dt
    test_dt = dt['math_data']
    #print(test_data)

tf_test_dt = test_dt.reshape((test_dt.shape[0]*4, 28, 28, 1)).astype('float32')
#print("TEST DATA : \n", tf_test_data)
tf_test_dt = tf_test_dt / 255

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 0)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 1)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 2)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 3)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 4)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 5)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 6)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 7)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 8)

In [ ]:
predict_and_compare(test_dt, tf_test_dt, 9)